# Prepare Dynamic Data Masking Demo
## Create a Database Master Key
Only necessary if one does not already exist.
Required to encrypt the credential secret in the next step.

In [1]:
CREATE MASTER KEY;

## Create a Database Scoped Credential and Data Source
### Accessing a Blob Storage with an Shared Access Signature

In [31]:
CREATE DATABASE SCOPED CREDENTIAL SharedAccessSignature
WITH
     IDENTITY = 'SHARED ACCESS SIGNATURE'
--   REMOVE ? FROM THE BEGINNING OF THE SAS TOKEN
,    SECRET = ''
;

In [40]:
CREATE EXTERNAL DATA SOURCE BlobStorage
WITH (
    TYPE = BLOB_STORAGE,
    LOCATION='https://<storage-account-name>.blob.core.windows.net/<container-name>', 
    CREDENTIAL = SharedAccessSignature
);

## Create Schema and Employee Table

In [12]:
CREATE SCHEMA HR;

In [66]:
CREATE TABLE [HR].Employees
(
    EmployeeName [nvarchar](100) NOT NULL
    , EmployeeNumber [int] NOT NULL
    , [State] [nvarchar](2) NULL
    , Zip [int] NULL
    , DOB [date] NULL
    , Age [int] NULL
    , Sex [nvarchar](10) NULL 
    , MaritalDesc [nvarchar](100) NULL
    , CitizenDesc [nvarchar](100) NULL
    , HispanicLatino [nvarchar](10) NULL 
    , RaceDesc [nvarchar](100) NULL
    , DateOfHire [date] NULL
    , DateOfTermination [date] NULL
    , ReasonForTerm [nvarchar](100) NULL
    , EmploymentStatus [nvarchar](100) NULL
    , Department [nvarchar](100) NULL
    , Position [nvarchar](100) NULL
    , PayRate [decimal](5,2) NULL
    , ManagerName [nvarchar](100) NULL
    , EmployeeSource [nvarchar](100) NULL
    , PerformanceScore [nvarchar](100) NULL
);

## Import Employee Data and Generate a SSN

> This [Kaggle Dataset](https://www.kaggle.com/rhuebner/human-resources-data-set) has been used for the employees table.

In [67]:
BULK INSERT [HR].Employees 
FROM 'core_dataset.csv'
WITH   
    ( 
    DATA_SOURCE = 'BlobStorage'
    , FORMAT = 'CSV' 
    , FIRSTROW = 2
    , LASTROW = 302
    );

In [0]:
ALTER TABLE [HR].Employees
ADD [SocialSecurityNumber] NCHAR(11) NOT NULL
CONSTRAINT Default_SSN
DEFAULT ('000-00-0000');

In [11]:
SET NOCOUNT ON;

WHILE EXISTS(SELECT * FROM [HR].Employees WHERE [SocialSecurityNumber] = '000-00-0000')
BEGIN
    UPDATE  [HR].Employees
    SET     [SocialSecurityNumber] = LTRIM(STR(FLOOR(RAND()*(999-100+1))+100)) + '-' + LTRIM(STR(FLOOR(RAND()*(99-10+1))+10))  + '-' + LTRIM(STR(FLOOR(RAND()*(9999-1000+1))+1000))
    WHERE   EmployeeNumber = 
            (SELECT TOP 1 
                    EmployeeNumber 
            FROM    [HR].Employees
            WHERE   [SocialSecurityNumber] = '000-00-0000');
END

SET NOCOUNT OFF;

In [12]:
SELECT TOP 10 * FROM [HR].Employees;

## Generate Fake Credit Card Numbers for Customers Table

In [4]:
ALTER TABLE [SalesLT].[Customer]
ADD [CreditCard] NVARCHAR(20) NULL;

In [15]:
UPDATE  [SalesLT].[Customer]
SET     [CreditCard] = STUFF(STUFF(STUFF(LEFT(CONVERT(NVARCHAR, ABS(CONVERT(BIGINT,CONVERT(BINARY(8), NEWID())))), 16),13,0,'-'),9,0,'-'),5,0,'-')